In [1]:
# !pip install python-docx

In [2]:
# !pip install docxtpl

# input docx

In [3]:
from docx import Document

doc = Document("input.docx")

doc

# All Tables

In [4]:
for i in range(len(doc.tables)):
    table = doc.tables[i]
    print("table number" , "    " , "col" , "  row")
    print("    ",i , "           ", len(table.columns[0].cells) ,"   "  ,len(table.rows[0].cells))
    print()
    for k in range(len(table.columns[0].cells)):
        for j in range(len(table.rows[0].cells)):
            print(table.rows[k].cells[j].text , end="   ||   ")
        print()
    print()
    print("======"*18)

table number      col   row
     0             6     4

number   ||   Lorem ipsum   ||   Lorem ipsum   ||   Lorem ipsum   ||   
1   ||   In eleifend velit vitae libero sollicitudin euismod.   ||   Lorem   ||      ||   
2   ||   Cras fringilla ipsum magna, in fringilla dui commodo a.   ||   Ipsum   ||      ||   
3   ||   Aliquam erat volutpat.    ||   Lorem   ||      ||   
4   ||   Fusce vitae vestibulum velit.    ||   Lorem   ||      ||   
5   ||   Etiam vehicula luctus fermentum.   ||   Ipsum   ||      ||   

table number      col   row
     1             6     2

number   ||   cost   ||   
1   ||   154.2   ||   
2   ||   578.1   ||   
3   ||   541.6   ||   
4   ||   21.2   ||   
5   ||   41.3   ||   



# All Pictures

In [5]:
from docx import Document
document = Document("input.docx")

with open("word.xml" , "w" , encoding="utf-8") as f:
    f.write(document._element.xml)

In [6]:
l_pics = []
with open("word.xml" , "r" , encoding="utf-8") as f:
    for l in f.readlines():
        if "wp:docPr id" in l:
            l_pics.append(l.split("\"")[-2])
            print(l.split("\"")[-2])
            print(l)
            print("====="*15)

print("number of pictures: " , len(l_pics))
print()
l_pics


            <wp:docPr id="1" name="Image2" descr=""/>


            <wp:docPr id="2" name="Image1" descr=""/>


            <wp:docPr id="3" name="Picture" descr=""/>

number of pictures:  3



['', '', '']

# output docx

In [7]:
# create a copy of input.docx
doc.save("output.docx")

### now we will change the tables and pictures of output.docx

### change tables

#### change first table

In [8]:
import pandas as pd
t0 = pd.read_excel("table0.xlsx")
t0

,number,Lorem ipsum,Lorem ipsum.1,Lorem ipsum.2
0,1,aaaaaaaa,ffffffffffff,lllllllllll
1,2,bbbbbbbb,ggggggg,mmm
2,3,ccccccccc,iiiiiiiiiiiiiiii,nnnnn
3,4,dddddddd,jjjjjjjjjjjjjjjj,ooooo
4,5,eeeeeeee,kkkkkkk,ppppp


In [9]:
from docx.table import Table
from docx.enum.table import  WD_ALIGN_VERTICAL 
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.shared import Pt
from docx.text.paragraph import Paragraph


table = doc.tables[0]

for k in range(1, len(table.columns[0].cells)):
    for j in range(1, len(table.rows[0].cells)):
        table.rows[k].cells[j].text = str(t0.iloc[k-1,j])
#         print(str(t0.iloc[k-1,j]))
        p = table.rows[k].cells[j].paragraphs[0]
        p.alignment = WD_ALIGN_PARAGRAPH.CENTER
        table.cell(k,j).vertical_alignment = WD_ALIGN_VERTICAL.CENTER
        paragraph_format = p.paragraph_format
#         paragraph_format.line_spacing = Pt(0)
#         paragraph_format.space_after = Pt(0)


#### change second table

In [10]:
import pandas as pd
t1 = pd.read_excel("table1.xlsx")
t1

,number,cost
0,1,150.3
1,2,598.6
2,3,536.1
3,4,19.6
4,5,45.1


In [11]:
from docx.table import Table
from docx.enum.table import  WD_ALIGN_VERTICAL 
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.shared import Pt
from docx.text.paragraph import Paragraph


table = doc.tables[1]

for k in range(1, len(table.columns[0].cells)):
    for j in range(1, len(table.rows[0].cells)):
        table.rows[k].cells[j].text = str(t1.cost.iloc[k-1])
        p = table.rows[k].cells[j].paragraphs[0]
        p.alignment = WD_ALIGN_PARAGRAPH.CENTER
        table.cell(k,j).vertical_alignment = WD_ALIGN_VERTICAL.CENTER
        paragraph_format = p.paragraph_format
#         paragraph_format.line_spacing = Pt(0)
#         paragraph_format.space_after = Pt(0)


In [12]:
doc.save("output.docx")


# Change Pictures

In [13]:
from docx import Document
document = Document("input.docx")

with open("word.xml" , "w" , encoding="utf-8") as f:
    f.write(document._element.xml)
l_pics = []
with open("word.xml" , "r" , encoding="utf-8") as f:
    for l in f.readlines():
        if "wp:docPr id" in l:
            l_pics.append(l[l.find('name="')+6 : l.find('descr')-2])
            print(l)
            print("====="*15)

print("number of pictures: " , len(l_pics))
print()
l_pics

            <wp:docPr id="1" name="Image2" descr=""/>

            <wp:docPr id="2" name="Image1" descr=""/>

            <wp:docPr id="3" name="Picture" descr=""/>

number of pictures:  3



['Image2', 'Image1', 'Picture']

### new pictures directories

In [14]:
# these pictures must be in your directory
new_pictures = ["barplot2.png" , "laptop_acer1.jpeg" , "laptop_acer2.jpeg"]

In [15]:
%%time

from docxtpl import DocxTemplate
from docx import Document


doc = DocxTemplate("output.docx")

context = {}
for old_pic , new_pic in zip(l_pics , new_pictures):
    doc.replace_pic(old_pic , new_pic)
    
doc.render(context)
doc.save("output.docx")

CPU times: user 101 ms, sys: 8.14 ms, total: 109 ms
Wall time: 109 ms
